<a href="https://colab.research.google.com/github/rikvik2006/MNIST-Challenge-PyTorch/blob/master/MNIST_Exercise_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installiamo le Dipendenze
Di default Google Collab ha gia installato le dipendenze che ci servono

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
import torch
from torchvision import datasets, transforms

